In [1]:
import psycopg2
import json
import csv
import pandas as pd
import psycopg2
from psycopg2 import sql
from psycopg2.extras import execute_values


In [2]:
def connect_to_postgres(dbname=None):
    """Connect to PostgreSQL. If dbname is None, connect to the default database."""
    return psycopg2.connect(
        dbname=dbname or "postgres",
        user="postgres",
        password="123456",
        host="localhost",
        port="5432"
    )

def create_database_if_not_exists(dbname):
    """Create the database if it does not exist."""
    conn = connect_to_postgres()
    conn.autocommit = True
    try:
        with conn.cursor() as cur:
            cur.execute(
                sql.SQL("CREATE DATABASE {};").format(sql.Identifier(dbname))
            )
            print(f"Database '{dbname}' created successfully.")
    except psycopg2.errors.DuplicateDatabase:
        print(f"Database '{dbname}' already exists.")
    finally:
        conn.close()

def create_tables_if_not_exists(conn):
    """Create tables in the database if they do not exist."""
    table_creation_queries = {
    "products": """
    CREATE TABLE IF NOT EXISTS products (
        product_id SERIAL PRIMARY KEY,
        product_name TEXT NOT NULL,
        price NUMERIC NOT NULL,
        category TEXT NOT NULL
    );
    """,
    "customers": """
    CREATE TABLE IF NOT EXISTS customers (
        customer_id SERIAL PRIMARY KEY,
        first_name TEXT NOT NULL,
        last_name TEXT NOT NULL,
        email TEXT NOT NULL,
        phone TEXT,
        address TEXT,
        city TEXT,
        gender TEXT,
        age INT
    );
    """,
    "orders": """
    CREATE TABLE IF NOT EXISTS orders (
        order_id SERIAL PRIMARY KEY,
        customer_id INT REFERENCES customers(customer_id),
        order_date DATE NOT NULL,
        payment_method TEXT,
        total_amount NUMERIC NOT NULL
    );
    """,
    "order_details": """
    CREATE TABLE IF NOT EXISTS order_details (
        order_detail_id SERIAL PRIMARY KEY,
        order_id INT REFERENCES orders(order_id),
        product_id INT REFERENCES products(product_id),
        quantity INT NOT NULL,
        subtotal NUMERIC NOT NULL
    );
    """
    }
    
    with conn.cursor() as cur:
        for table, query in table_creation_queries.items():
            cur.execute(query)
        conn.commit()
    print("Tables created successfully (if they did not already exist).")

def insert_data_to_postgres(conn, table_name, data):
    """Insert data into a PostgreSQL table."""
    with conn.cursor() as cur:
        columns = data.columns.tolist()
        values = [tuple(x) for x in data.to_numpy()]
        insert_query = f"""
        INSERT INTO {table_name} ({', '.join(columns)}) 
        VALUES %s
        """
        execute_values(cur, insert_query, values)
        conn.commit()

In [3]:
# Define the database name
db_name = "supermarket"

# Create the database if it doesn't exist
create_database_if_not_exists(db_name)

# Connect to the newly created database
conn = connect_to_postgres(dbname=db_name)

# Ensure tables exist
create_tables_if_not_exists(conn)


Database 'supermarket' created successfully.
Tables created successfully (if they did not already exist).


In [4]:
# Load CSV files into DataFrames
customers_df = pd.read_csv('customers.csv')
order_details_df = pd.read_csv('order_details.csv')
orders_df = pd.read_csv('orders.csv')
products_df = pd.read_csv('products.csv')

In [5]:
try:
    # Transfer data to PostgreSQL
    insert_data_to_postgres(conn, "products", products_df)
    insert_data_to_postgres(conn, "customers", customers_df)
    insert_data_to_postgres(conn, "orders", orders_df)
    insert_data_to_postgres(conn, "order_details", order_details_df)
    print("Data successfully transferred to PostgreSQL database.")
finally:
    conn.close()

Data successfully transferred to PostgreSQL database.
